In [12]:
import pandas as pd
import os

In [13]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/raw_data/'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned_data/'
variables_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/variables/'

In [52]:
xl = pd.ExcelFile(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx')
res = len(xl.sheet_names) #Number of sheets

df = pd.read_excel(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx',
                   sheet_name=0)

In [31]:
frims_rc_id_mapping = pd.read_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/frims_rc_id_mapping.csv')
frims_rc_id_mapping = frims_rc_id_mapping[['object_id','RC_Id']]

In [56]:
df = df.merge(frims_rc_id_mapping, on='RC_Id', how='left')

In [60]:
unmapped_rcs = set(df[df['object_id'].isnull()]['RC_Id'].dropna().to_list())
unmapped_rcs

set()

In [61]:
cols = df.columns.to_list()

In [62]:
cols_to_remove_for_aggregation = ['Id','Report_Date','Report Date',
                                  'District_Id','District Id',
                                  'Revenue_Name','Revenue Circle Name',
                                  'RC_Id', 'Revenue Circle Id', 
                                  'District_Name', 'District Name',
                                  'DFC_Id','DCF_Id', 'object_id',
                                  'Block_Name',
                                  'Type', 'Name','Location',''
                                 'Relief_Distribution_Center_Name', 'Child_friendly_Space',
                                 'Latitude', 'Longitude',
                                  'Relief_Distribution_Other_Item_Name', 'Relief_Distribution_Other_Item_Qty',
                                 'Relief_Distribution_Other_Item_Unit',
                                 'Human_Live_Lost_Type']

for col_to_remove in cols_to_remove_for_aggregation:
    try:
        cols.remove(col_to_remove)
    except:
        continue

In [63]:
grouped_df = df.groupby(['Report_Date', 'object_id'])[cols].sum()
grouped_df

Population_affected_Male  Population_affected_Female  \
Report_Date object_id                                                         
2021-05-15  101.0                           0.0                         0.0   
            102.0                           0.0                         0.0   
            103.0                           0.0                         0.0   
            104.0                           0.0                         0.0   
            105.0                           0.0                         0.0   
...                                         ...                         ...   
2023-08-30  277.0                           0.0                         0.0   
            278.0                           0.0                         0.0   
            279.0                           0.0                         0.0   
            280.0                        5768.0                      5423.0   
            281.0                       20151.0                     18857.0   

                       Population_affected_Total  House_Fully_Damaged_Kuccha  \
Report_Date object_id                                                          
2021-05-15  101.0                            0.0                         0.0   
            102.0                            0.0                         0.0   
            103.0                            0.0                         0.0   
            104.0                            0.0                         0.0   
            105.0                            0.0                         0.0   
...                                          ...                         ...   
2023-08-30  277.0                            0.0                         0.0   
            278.0                            0.0                         0.0   
            279.0                            0.0                         0.0   
            280.0                        12506.0                         0.0   
            281.0                        52529.0                         0.0   

                       House_Fully_Damaged_Pukka  Total_House_Fully_Damaged  \
Report_Date object_id                                                         
2021-05-15  101.0                            0.0                        0.0   
            102.0                            0.0                        0.0   
            103.0                            0.0                        0.0   
            104.0                            0.0                        0.0   
            105.0                            0.0                        0.0   
...                                          ...                        ...   
2023-08-30  277.0                            0.0                        0.0   
            278.0                            0.0                        0.0   
            279.0                            0.0                        0.0   
            280.0                            0.0                        0.0   
            281.0                            0.0                        0.0   

                       House_Partially_Damaged_Kuccha  \
Report_Date object_id                                   
2021-05-15  101.0                                 0.0   
            102.0                                 0.0   
            103.0                                 0.0   
            104.0                                 0.0   
            105.0                                 0.0   
...                                               ...   
2023-08-30  277.0                                 0.0   
            278.0                                 0.0   
            279.0                                 0.0   
            280.0                                 0.0   
            281.0                                 0.0   

                       House_Partially_Damaged_Pukka  \
Report_Date object_id                                  
2021-05-15  101.0                                0.0   
            102.

In [64]:
variables = grouped_df.columns

In [65]:

for variable in variables:
    variable_df = grouped_df[variable].reset_index()
    variable_df['year'] = variable_df.Report_Date.dt.year
    variable_df['month'] = variable_df.Report_Date.dt.month
    variable_df['month'] = variable_df.month.map("{:02}".format)
    variable_df = variable_df.groupby(['year','month','object_id'])[variable].sum().reset_index()
    variable_df['year_month'] = variable_df['year'].astype(str) + '_' + variable_df['month'].astype(str)
    
    for year_month in variable_df['year_month'].unique():
        variable_df_monthly = variable_df[variable_df['year_month']==year_month]        
        
        variable_df_monthly = variable_df_monthly[['object_id', variable]]
        if os.path.exists(variables_path+variable):
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
        else:
            os.mkdir(variables_path+variable)
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)

In [ ]:
df[df['RC_Id']==188]

In [ ]:
# FRIMS RC ID MAPPING

In [ ]:
import geopandas as gpd
rc_gdf = gpd.read_file('/home/krishna/IDS-DRR-Data-Pipeline/Maps/Assam_Revenue_Circles/assam_revenue_circle_nov2022.geojson')

In [ ]:
rc_gdf = rc_gdf[['revenue_ci', 'object_id','district_3']]

In [ ]:
frims_rcs = df[['Revenue_Name','RC_Id','District_Name']].drop_duplicates()
frims_rcs

In [ ]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
fuzzymatch = fuzzy_merge(rc_gdf, frims_rcs, 'revenue_ci', 'Revenue_Name', threshold=80,limit=1)
fuzzymatch

In [ ]:
fuzzymatch.merge(frims_rcs, left_on='matches', right_on='Revenue_Name').to_csv('frims_rc_id_mapping.csv', index=False)